# Árbol de Segmentos para Varianzas de Intervalos

## Implementación Completa en Python

Este notebook contiene la implementación completa de un árbol de segmentos que mantiene varianzas usando la **fórmula compacta**:

$$Var(X) = E[X^2] - (E[X])^2$$

### Contenido:
1. Clases Node y SegmentTreeVariance
2. Función de visualización gráfica
3. Ejemplos prácticos del mundo real

## 1. Implementación del Árbol de Segmentos

In [ ]:
import random

class Node:
    """Nodo del árbol que almacena: suma, suma_cuadrados, count"""
    
    def __init__(self, suma=0, suma_cuadrados=0, count=0):
        self.suma = suma
        self.suma_cuadrados = suma_cuadrados
        self.count = count

    def get_variance(self):
        """Calcula varianza: Var(X) = E[X²] - (E[X])²"""
        if self.count == 0:
            return 0
        media = self.suma / self.count
        media_cuadrados = self.suma_cuadrados / self.count
        return max(0, media_cuadrados - media * media)

    def get_mean(self):
        return self.suma / self.count if self.count > 0 else 0

    def __repr__(self):
        return f"Node(Σ={self.suma}, Σ²={self.suma_cuadrados}, n={self.count}, var={self.get_variance():.2f})"


class SegmentTreeVariance:
    """Árbol de Segmentos para varianzas. O(n) construcción, O(log n) consultas/actualizaciones"""

    def __init__(self, arr):
        self.arr = arr[:]
        self.n = len(arr)
        self.tree = [Node() for _ in range(4 * self.n)]
        if self.n > 0:
            self._build(0, 0, self.n - 1)

    def _merge(self, left, right):
        """Combina dos nodos hijos"""
        return Node(left.suma + right.suma, 
                   left.suma_cuadrados + right.suma_cuadrados, 
                   left.count + right.count)

    def _build(self, idx, l, r):
        if l == r:
            val = self.arr[l]
            self.tree[idx] = Node(val, val*val, 1)
            return
        mid = (l + r) // 2
        self._build(2*idx+1, l, mid)
        self._build(2*idx+2, mid+1, r)
        self.tree[idx] = self._merge(self.tree[2*idx+1], self.tree[2*idx+2])

    def update(self, index, new_value):
        """Actualiza arr[index] = new_value"""
        if 0 <= index < self.n:
            self.arr[index] = new_value
            self._update_rec(0, 0, self.n-1, index, new_value)

    def _update_rec(self, idx, l, r, target, val):
        if l == r:
            self.tree[idx] = Node(val, val*val, 1)
            return
        mid = (l + r) // 2
        if target <= mid:
            self._update_rec(2*idx+1, l, mid, target, val)
        else:
            self._update_rec(2*idx+2, mid+1, r, target, val)
        self.tree[idx] = self._merge(self.tree[2*idx+1], self.tree[2*idx+2])

    def query_variance(self, ql, qr):
        """Consulta varianza en rango [ql, qr]"""
        return self._query(0, 0, self.n-1, ql, qr).get_variance()

    def query_mean(self, ql, qr):
        """Consulta media en rango [ql, qr]"""
        return self._query(0, 0, self.n-1, ql, qr).get_mean()

    def query_sum(self, ql, qr):
        """Consulta suma en rango [ql, qr]"""
        return self._query(0, 0, self.n-1, ql, qr).suma

    def _query(self, idx, l, r, ql, qr):
        if qr < l or ql > r:  # Sin solapamiento
            return Node()
        if ql <= l and r <= qr:  # Solapamiento total
            return self.tree[idx]
        mid = (l + r) // 2
        return self._merge(self._query(2*idx+1, l, mid, ql, qr),
                         self._query(2*idx+2, mid+1, r, ql, qr))

    def get_array(self):
        return self.arr[:]

print("✓ Clases Node y SegmentTreeVariance cargadas")

## 2. Función de Visualización Gráfica

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math

%matplotlib inline

def visualizar_arbol(arr, mostrar=True, guardar=None):
    """Genera visualización gráfica del árbol de segmentos"""
    st = SegmentTreeVariance(arr)
    n = len(arr)
    height = math.ceil(math.log2(n)) + 1 if n > 0 else 1
    
    fig_width = max(12, n * 1.5)
    fig_height = max(8, height * 2)
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    def draw_node(idx, l, r, x, y, level):
        if l > r or idx >= len(st.tree):
            return
        
        node = st.tree[idx]
        w, h = 2.5, 1.2
        color = '#FFE8E8' if l == r else '#E8F4F8'
        
        rect = patches.FancyBboxPatch((x-w/2, y-h/2), w, h, 
                                     boxstyle="round,pad=0.1", linewidth=2,
                                     edgecolor='#2C3E50', facecolor=color)
        ax.add_patch(rect)
        
        text = f"[{l},{r}]\nΣ={node.suma:.0f}\nΣ²={node.suma_cuadrados:.0f}\nVar={node.get_variance():.2f}"
        ax.text(x, y, text, ha='center', va='center', fontsize=9, fontweight='bold')
        
        if l < r:
            mid = (l + r) // 2
            spacing = fig_width / (2 ** (level + 2))
            lx, rx = x - spacing, x + spacing
            cy = y - 2.5
            
            ax.plot([x, lx], [y-h/2, cy+h/2], linewidth=2, color='#34495E')
            ax.plot([x, rx], [y-h/2, cy+h/2], linewidth=2, color='#34495E')
            
            draw_node(2*idx+1, l, mid, lx, cy, level+1)
            draw_node(2*idx+2, mid+1, r, rx, cy, level+1)
    
    ay = height * 2.5
    ax.text(fig_width/2, ay+1, f"Array: {arr}", ha='center', fontsize=12, fontweight='bold',
           bbox=dict(boxstyle='round', facecolor='#FFF9E6', edgecolor='#F39C12', linewidth=2))
    
    draw_node(0, 0, n-1, fig_width/2, ay-2, 0)
    
    ax.text(fig_width/2, ay+2.5, 'Árbol de Segmentos - Varianzas', 
           ha='center', fontsize=16, fontweight='bold')
    ax.text(1, -1, 'Σ = Suma\nΣ² = Suma cuadrados\nVar = Varianza',
           ha='left', va='top', fontsize=10,
           bbox=dict(boxstyle='round', facecolor='#F8F9FA', edgecolor='#95A5A6'))
    
    ax.set_xlim(0, fig_width)
    ax.set_ylim(-2, ay+3)
    ax.axis('off')
    plt.tight_layout()
    
    if guardar:
        plt.savefig(guardar, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"✓ Guardado: {guardar}")
    if mostrar:
        plt.show()
    else:
        plt.close()

print("✓ Función visualizar_arbol() cargada")

## 3. Ejemplos Prácticos

### Ejemplo 1: Visualización del Árbol

In [ ]:
# Visualizar un array pequeño
visualizar_arbol([4, 8, 6, 2])

### Ejemplo 2: Consultas Básicas

In [ ]:
arr = [4, 8, 6, 2, 10, 12, 14, 16]
st = SegmentTreeVariance(arr)

print(f"Array: {arr}\n")
print("Consultas de varianza:")
for l, r in [(0, 3), (2, 5), (0, 7), (4, 7)]:
    var = st.query_variance(l, r)
    mean = st.query_mean(l, r)
    print(f"  [{l},{r}]: Media={mean:.2f}, Varianza={var:.4f}")

### Ejemplo 3: Análisis de Temperaturas de Sensores

In [ ]:
temperaturas = [18, 17, 16, 15, 16, 18, 20, 22, 25, 27, 29, 30,
                31, 32, 31, 30, 28, 26, 24, 22, 21, 20, 19, 18]

st_temp = SegmentTreeVariance(temperaturas)

print("Análisis de Temperatura por Períodos del Día\n" + "="*50)

periodos = [("Madrugada (0-5h)", 0, 5), ("Mañana (6-11h)", 6, 11), 
            ("Tarde (12-17h)", 12, 17), ("Noche (18-23h)", 18, 23)]

for nombre, inicio, fin in periodos:
    media = st_temp.query_mean(inicio, fin)
    var = st_temp.query_variance(inicio, fin)
    std = var ** 0.5
    print(f"{nombre:18s}: Media={media:5.2f}°C, Var={var:6.4f}, Desv.Est={std:5.2f}°C")

### Ejemplo 4: Análisis de Calificaciones

In [ ]:
calificaciones = [85, 90, 78, 92, 88, 76, 95, 89, 91, 87, 82, 94, 88, 90, 86, 93, 79, 91, 88, 84]
st_calif = SegmentTreeVariance(calificaciones)

print(f"Calificaciones de {len(calificaciones)} estudiantes\n" + "="*50)
print(f"Media general: {st_calif.query_mean(0, len(calificaciones)-1):.2f}")
print(f"Varianza: {st_calif.query_variance(0, len(calificaciones)-1):.4f}")
print(f"Desviación estándar: {st_calif.query_variance(0, len(calificaciones)-1)**0.5:.2f}")

print("\nPor grupos de 5 estudiantes:")
for i in range(0, len(calificaciones), 5):
    fin = min(i+4, len(calificaciones)-1)
    media = st_calif.query_mean(i, fin)
    var = st_calif.query_variance(i, fin)
    print(f"  Grupo {i//5+1}: Media={media:.2f}, Varianza={var:.4f}")

### Ejemplo 5: Volatilidad Financiera

In [ ]:
precios = [100, 102, 101, 103, 105, 104, 106, 108, 107, 109,
           111, 110, 112, 115, 114, 116, 115, 117, 119, 118,
           120, 122, 121, 123, 125, 124, 126, 128, 127, 129]

st_precios = SegmentTreeVariance(precios)

print("Análisis de Volatilidad Semanal (5 días hábiles)\n" + "="*50)

for sem in range(6):
    inicio = sem * 5
    fin = min(inicio + 4, len(precios) - 1)
    if inicio >= len(precios):
        break
    
    media = st_precios.query_mean(inicio, fin)
    var = st_precios.query_variance(inicio, fin)
    volatilidad = (var ** 0.5) / media * 100
    
    print(f"Semana {sem+1}: Precio=${media:.2f}, Var={var:.2f}, Volatilidad={volatilidad:.2f}%")

### Ejemplo 6: Actualizaciones Dinámicas

In [ ]:
arr_dinamico = [10, 20, 30, 40, 50]
st_din = SegmentTreeVariance(arr_dinamico)

print("Efecto de Actualizaciones en la Varianza\n" + "="*50)
print(f"Array inicial: {st_din.get_array()}")
print(f"Varianza: {st_din.query_variance(0, 4):.4f}\n")

# Hacer el array más uniforme
updates = [(0, 30), (4, 30)]
for idx, val in updates:
    st_din.update(idx, val)
    print(f"Después update({idx}, {val}): {st_din.get_array()}")
    print(f"  -> Varianza: {st_din.query_variance(0, 4):.4f}\n")

## 4. Experimenta con tus Propios Datos

Modifica el array abajo y ejecuta la celda para ver la visualización:

In [ ]:
# ¡Modifica este array con tus propios valores!
mi_array = [15, 8, 12, 6, 20, 3]

# Crear árbol y consultar
st_custom = SegmentTreeVariance(mi_array)

print(f"Tu array: {mi_array}")
print(f"Media: {st_custom.query_mean(0, len(mi_array)-1):.2f}")
print(f"Varianza: {st_custom.query_variance(0, len(mi_array)-1):.4f}\n")

# Visualizar
visualizar_arbol(mi_array)

## 5. Resumen y Conclusiones

### Complejidades:
- **Construcción**: O(n)
- **Consulta (varianza/media/suma)**: O(log n)
- **Actualización**: O(log n)

### Fórmula Clave:
$$Var(X) = E[X^2] - (E[X])^2 = \frac{\sum x_i^2}{n} - \left(\frac{\sum x_i}{n}\right)^2$$

### Ventajas:
- Permite combinar rangos eficientemente
- Solo necesita: suma, suma de cuadrados y conteo
- Ideal para análisis estadístico dinámico